In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [0]:
# Load per cycle data frame
import pickle
folder = "/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/" 
filename = folder + 'w_c_dataset.pickle'
infile = open(filename,'rb')
[sound,sr,lengths,times,labels] = pickle.load(infile)
infile.close()

In [0]:
def split_sounds(sounds,times,labels):
    s=[]
    l=[]
    for i,sound in enumerate(sounds):
        for t,label in zip(times[i],labels[i]):
            s.append(sound[int(t[0]):int(t[1])])
            if label==0:
                a=np.array([1,0,0,0])
            if label==1:
                a=np.array([0,1,0,0])
            if label==2:
                a=np.array([0,0,1,0])    
            if label==3:
                a=np.array([0,0,0,1])
            l.append(a)
    return s,l

In [0]:
[data,label]=split_sounds(sound,times,labels)

In [0]:
# Extract chroma feature from the saved file
base_path = "/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/Chroma-smooth/RC/"
max_pad_len = 697
import librosa
def extract_feature(cycle_number,X,sample_rate):
    full_file_path = base_path + str(cycle_number) + '.csv'
    chroma_vec = pd.read_csv(full_file_path,header=None).to_numpy()
    mean_chroma = np.mean(chroma_vec.T,axis=0)
    std_chroma = np.std(chroma_vec.T,axis=0)
    check = np.hstack((mean_chroma,std_chroma))
    return check

In [32]:
import sys
dataset = []

for cycle_number,d in enumerate(data):
    a = extract_feature(cycle_number+1,d,sr[0])
    dataset.append(a)
    if(cycle_number%1000 == 0):
        print(cycle_number)

dataset = np.array(dataset)
print(dataset.shape)

0
1000
2000
3000
4000
5000
6000
(6898, 24)


In [33]:


label=np.asarray(label)
a=np.zeros(label.shape[0])
for i in range(label.shape[0]):
    for j in range(label.shape[1]):
        if label[i][j]==1:
            a[i]=j

# COnvert to binary for the binary classification problem
bin_label =np.zeros(label.shape[0])
for index,l in enumerate(a):
    if l == 0:
        bin_label[index] = l
    else:
        bin_label[index] = 1

unique, counts = np.unique(bin_label, return_counts=True)

print(np.asarray((unique, counts)).T)


# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(bin_label)
oh_labels = to_categorical(i_labels)

[[0.000e+00 3.642e+03]
 [1.000e+00 3.256e+03]]


In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(dataset,i_labels,test_size=0.3, random_state=42,stratify=i_labels)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [35]:
Cs = [2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10)]
gamma = [2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]


param_grid = {'C': Cs,  
                'gamma': gamma, 
                'kernel': ['rbf'],
                'class_weight': ['balanced']}  



grid1 = GridSearchCV(SVC(), param_grid,cv=3,n_jobs=-1, verbose = 3) 

# fitting the model for grid search 
grid1.fit(x_train, y_train)

# print best parameter after tuning 
print("Best Parameters after tuning:")
print(grid1.best_params_) 
# print how our model looks after hyper-parameter tuning
print("Best selected Model: ") 
print(grid1.best_estimator_)

grid_predictions = grid1.predict(x_test) 

# print classification report
print("Classification Report: ") 
print(classification_report(y_test, grid_predictions))
print("Classification Accuracy: ")
print(accuracy_score(y_test,grid_predictions))

print("Confusion Matrix is as Follows: ")
print(sklearn.metrics.confusion_matrix(y_test,grid_predictions))

Fitting 3 folds for each of 143 candidates, totalling 429 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 429 out of 429 | elapsed:  8.1min finished


Best Parameters after tuning:
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'}
Best selected Model: 
SVC(C=1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Classification Report: 
              precision    recall  f1-score   support

           0       0.54      0.79      0.64      1093
           1       0.52      0.26      0.35       977

    accuracy                           0.54      2070
   macro avg       0.53      0.52      0.49      2070
weighted avg       0.53      0.54      0.50      2070

Classification Accuracy: 
0.538647342995169
Confusion Matrix is as Follows: 
[[863 230]
 [725 252]]


In [36]:
# Running XgBoost
import xgboost as xgb

print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=3,num_class=2, n_estimators=15, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)
print(accuracy_score(y_test,xgb_pred))

Training XGB Classifier from new features:
0.5560386473429951


In [39]:
# One hot the labels
le = LabelEncoder()
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)



# Custom model for classification
model = Sequential()

model.add(Dense(24, activation='relu', input_shape=(24,),kernel_initializer='random_normal'))

model.add(Dense(12, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(300, activation='relu'))

#model.add(Dense(150, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(75, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(64, activation='relu',kernel_initializer='random_normal')) 
#model.add(Dropout(0.2))


#model.add(Dense(16, activation='relu') )

model.add(Dense(2, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

model.summary()



# Load the trained model if required
#model_file = '/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/LogMel_MFCC_Chroma_CNN/Try6_Unlinked/model_40_230.h5'
#model.load_weights(model_file)

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_4 (Dense)              (None, 12)                300       
_________________________________________________________________
dropout_1 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 26        
Total params: 926
Trainable params: 926
Non-trainable params: 0
_________________________________________________________________
65/65 [==============================] - 0s 1ms/step - loss: 0.6970 - accuracy: 0.4812
Pre-training accuracy: 48.1159%


In [40]:
#train model
num_epochs = 1000
num_batch_size = 128



callbacks = [
    ModelCheckpoint(
        filepath='/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/Chroma_ANN_Binary/Chroma_ANN_Binary_{epoch:02d}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), callbacks=callbacks, verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/1000
29/38 [=====================>........] - ETA: 0s - loss: 0.6943 - accuracy: 0.5057
Epoch 00001: val_accuracy improved from -inf to 0.53623, saving model to /content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/Chroma_ANN_Binary/Chroma_ANN_Binary_01.h5
38/38 [==============================] - 0s 4ms/step - loss: 0.6937 - accuracy: 0.5095 - val_loss: 0.6900 - val_accuracy: 0.5362
Epoch 2/1000
36/38 [===========================>..] - ETA: 0s - loss: 0.6908 - accuracy: 0.5347
Epoch 00002: val_accuracy improved from 0.53623 to 0.54493, saving model to /content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/Chroma_ANN_Binary/Chroma_ANN_Binary_02.h5
38/38 [==============================] - 0s 4ms/step - loss: 0.6906 - accuracy: 0.5385 - val_loss: 0.6889 - val_accuracy: 0.5449
Epoch 3/1000
36/38 [===========================>..] - ETA: 0s - loss: 0.6891 - accuracy: 0.5469
Epoch 00003: val_accuracy improved from 0